In [1]:
import os
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.externals import joblib

warnings.filterwarnings(action='ignore');
from utils import getCode

c:\users\youjeongsue\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
#파일 저장후 다시 읽어와야 할 때
env_cultivate_mean_data_minmax = pd.read_csv('./data/data_for_dashboard.csv', index_col=0)

del_code=['growLength', 'flowerTop', 'stemDiameter','leavesWidth', 'leavesNum',
 'flowerPosition', 'fruitsPosition', 'index', 'measDate', 'userId']

for code in del_code:
    del env_cultivate_mean_data_minmax[code]

In [7]:
columns=list(env_cultivate_mean_data_minmax.columns)
columns.remove('leavesLength')
columns.remove('fruitsNum')

In [17]:
columns

['WS', 'EL', 'CI', 'TE', 'WD', 'HI', 'TL', 'IR', 'HL', 'TI', 'RP']

In [8]:
from sklearn.model_selection import RepeatedKFold

def printScores(model, x, y):
    print("R^2: ",model.score(x, y))
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    mae = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    mae = np.absolute(mae)
    rmse = cross_val_score(model, x, y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
    rmse = np.absolute(rmse)
    print('Mean MAE: %.3f (%.3f)' % (mean(mae), std(mae)))
    print('Mean RMSE: %.3f' % (mean(rmse)))

In [20]:
from sklearn.model_selection import train_test_split

x=env_cultivate_mean_data_minmax[columns[:]]
y_leaves=env_cultivate_mean_data_minmax['leavesLength']
y_fruits=env_cultivate_mean_data_minmax['fruitsNum']

x_train_l, x_test_l, y_train_l, y_test_l = train_test_split(x, y_leaves, test_size=0.2)
x_train_f, x_test_f, y_train_f, y_test_f = train_test_split(x, y_fruits, test_size=0.2)

In [21]:
x

,WS,EL,CI,TE,WD,HI,TL,IR,HL,TI,RP
0,1.198452,9.841488,632.278690,6.910179,42.461131,88.061131,18.460000,127.018452,95.621488,17.460179,0.133750
1,1.409581,9.839581,625.905749,8.269042,42.085988,87.423533,17.159401,138.891078,96.254192,16.569521,0.078922
2,1.726706,9.809921,615.776548,13.114960,39.648532,88.142480,17.982619,183.277321,94.930833,17.895655,0.074504
3,1.703512,9.803036,606.192381,15.772024,38.516369,93.215387,19.006399,231.922946,95.204137,18.671637,0.044673
4,1.482202,9.790119,596.609702,16.599405,39.730536,95.701190,18.536667,158.558869,95.001250,18.583631,0.128810
...,...,...,...,...,...,...,...,...,...,...,...
749,2.874461,9.178024,379.765060,14.605659,162.007036,97.749192,0.000000,234.871916,98.042156,19.261287,4.603413
750,1.778302,9.190503,385.764088,11.647610,243.391321,97.646164,0.000000,265.213270,97.974151,19.785220,0.013648
751,2.253375,9.179750,380.000000,18.099688,165.191500,98.093500,0.000000,279.171437,97.911375,21.334500,0.076250
752,2.412126,9.166257,379.378683,17.873743,172.880090,97.841048,0.000000,331.280719,97.661946,20.560359,14.472455


## RF_only_env

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [22]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

rf = RandomForestRegressor()

model_cv = GridSearchCV(estimator = rf, param_grid = param_grid, 
                        cv = 3, n_jobs = -1, verbose = 2,
                        scoring='r2',
                        return_train_score=True)

model_cv.fit(x_train_f, y_train_f)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  8.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [23]:
model_cv.best_params_

{'bootstrap': True,
 'max_depth': 110,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 200}

In [25]:
clf=RandomForestRegressor(bootstrap= True,
 max_depth=110,
 max_features=3,
 min_samples_leaf=3,
 min_samples_split=8,
 n_estimators=200)
clf.fit(x_train_f, y_train_f)

printScores(clf, x_test_f, y_test_f)

R^2:  0.7643075142463444
Mean MAE: 15.803 (3.111)
Mean RMSE: 21.772


In [26]:
joblib.dump(clf, 'RF_only_env.pkl')

['RF_only_env.pkl']